# Milestone 3 - Implementing Google’s Deep Dream

__Objective__: Implement Google’s Deep Dream algorithm. This allows you to start manipulating CNNs and understanding how we begin to use Artificial Intelligence to generate art by utilizing pre-learned knowledge stored in the latent space of pretrained CNNs.

__Workflow__:

The principle of Google's Deep Dream is that we freeze the weights of a pretrained CNN and instead of training, we modify the input image. It essentially creates a  feedback loop where the input image is fed back into the CNN, the CNN runs in reverse, and the output effects are amplified on our image.

1.  Create some helper utilities such as:
    - An image loader that converts a loaded image into a `NumPy` array in the RBG format 
    - A deprocess function that undoes the processing done for the Inception model.
2. Now let's calculate the loss for each iteration of the Deep Dream algorithm.
    -  We firstly add a batch dimension to the image and then grab the activations, from specified layers of the Inception network after performing a forward pass.
    - We will store the intermediate losses in a list and while iterating over the layer activations, we compute the mean of each activation.
3. Create a function that does a single pass of the deep dream algorithm. It is to perform a forward-pass and then taking the gradients, use them to update the input image to generate the deep dream effects.
4. Create another function that runs the previous function for a user-specified number of iterations (100 is a good default value). It takes in the image input, the model (inception preferably), and a step size setting (use 0.01 as the default).
    - In this function, we'll take the input image, apply our preprocess function to it
    - Next, we will iterate using our single-pass deep dream function to update the image.
    - We will then return the deproessed image.

5. Now let's build our deep dream model
    - Load the inception model using the pretrained imagenet weights and without the top included.
    - Choose the layers you'll be using to generate your deep dream effects e.g. `mixed3`, 'mixed7`, and so on. 
    - Set the number of octaves (i.e. the image sizes we'll be using).
    - Set the octave scale which is the scaling factor by which we change the image size used for each octave.
    - Create our `deepDreamModel` by using `tf.keras.Model(inputs=baseModel.input, outputs=layers)`
    - Convert the image to a TensorFlow constant for improved performance
    - Take the first two dimensions of the image and cast them to a `float`
6. Use all our building blocks to generate Deep Dream's effects
    - Iterate over the number of octaves
    - get the new image shape for that given octave and convert it to a NumPy array
    - run it through your full deep dream model that you created in step 10 

7. Save and display your image. Experiment with different octaves, scales, layers to produce some trippy effects! Congratulations, you've just made your first AI-generated Art. Next, we'll step this up a notch by attempting Neural Style Transfer in the next milestone. 

__Purpose__:

The purpose of this lesson was to demonstrate how we can create the building blocks and implement the tecniques used or Google Dream algorithm.
You would have learned how to use TensorFlow 2.0’s Gradient Tape to log gradients.  How to take the pretrained InceptionV3 model to create the Deep Dream Model. How the Deep Dream algorithm works by freezing the weights of a pretrained CNN and instead of training, we modify the input image. This essentially creates a feedback loop where the input image is fed back into the CNN, the CNN runs in reverse, and the output effects are amplified on our image.


__Deliverable__:

The deliverable is a Jupyter Notebook documenting your workflow as you implement Google's Deep Dream algorithm and share some of your AI generated Art by changing the number of Octaves, Octave scale, and layers used.

In [2]:
# Import the libraries we need to start putting Google's Deep Dream together
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from PIL import Image
import tensorflow as tf
import numpy as np
import imutils
import cv2

### Image Loader Function
Load the image, resize it to a width of 350. We then swap the color channels from BGR to RGB, and return the image as a NumPy array.

In [3]:
def loadImage(imagePath):
    '''returns the image pixel array resiezd to a width of 350'''
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=350)
    # convert from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.array(image)
    return image

### Deprocess Function 
Utility function to convert a tensor into a valid image. It "undoes" the preprocessing done for Inception and then casts the pixel values to integers

In [4]:
def deprocess(image):
    '''returns the deprocessed image'''
    image = 255 * (image + 1.0) 
    image /= 2.0
    image = tf.cast(image, tf.uint8)
    return image

### Calculate the loss after a single iteration of the DeepDream algorithm

We firstly add a batch dimension to the image and then grab the activations, from specified layers of the Inception network after performing a forward pass.

In [5]:
def calculateLoss(image, model):
    '''returns the sum of losses'''
    image = tf.expand_dims(image, axis=0)
    layerActivations = model(image)

    # Create a list to store the intermediate losses
    losses = []

    # iterate over the layer activations
    for act in layerActivations:
        # compute the mean of each activation
        loss = tf.reduce_mean(act)
        # append these losses to the losses list
        losses.append(loss)

    # return the sum of the losses
    return tf.reduce_sum(losses)

## The Deep Dream function

This function performs a forward-pass and then taking the
gradients and using them to update the input image to generate the deep dream effects.

We input our model (inception network preferablly), the input image, the step size we use for our gradient updates and EPS, a tiny value used to prevent divide by zero errors.

In [6]:
@tf.function
def deepDreamSinglePass(model, image, stepSize, eps=1e-8):
    ''' returns a tuple of the loss and the updated image'''

    # Tells TenorFlow to record gradients
    with tf.GradientTape() as tape:
        # keep track of the image to calculate gradients and loss 
        tape.watch(image)
        loss = calculateLoss(image, model)

    # calculates the gradients of the loss with respect to the image
    gradients = tape.gradient(loss, image)
    # normalize the gradients 
    gradients /= tf.math.reduce_std(gradients) + eps 
    # K.maximum(K.mean(K.abs(grads)), K.epsilon())

    # adjusts the image with the normalized gradients 
    image = image + (gradients * stepSize)
    # clip its pixel values to the range [-1, 1]
    image = tf.clip_by_value(image, -1, 1)

    return (loss, image)

### Function that runs our Deep Dream Model

Up to this point we can perform a single forward-pass of the DeepDream algorithm. We now need to design our training loop that enables us to do this for multiple iterations.

The `fullDeepDreamModel` we will be making below will allow us to pass our model (inception) and input image for a specified number of iterations.

In [7]:
def fullDeepDreamModel(model, image, iterations=50, stepSize=0.01):
    '''runs the deep dream algorithm for a specified number of iterations 
    and returns the depresocessed image'''

    # preprocess the image for the Inception network
    image = preprocess_input(image)

    # We iterate for the specified number of iterations
    for iteration in range(iterations):
        # use  our deepDreamSinglePass function to get the loss and the updated image
        (loss, image) = deepDreamSinglePass(model, image, stepSize)

        # print updates on the iteration progress
        if iteration % 50 == 0:
            print ("Iteration {} with loss {}".format(iteration, loss))

    return deprocess(image)

#### Define the layers we are going to use for the dream


In [8]:
# Chose the layers you'll be using 
names = ["mixed2", "mixed3","mixed4","mixed5", "mixed7"]

# define the octave scale and number of octaves (tweaking these values
# will produce different output dreams)
OCTAVE_SCALE = 1.3
NUM_OCTAVES = 6

### Execute Deep Dream



In [9]:
# get our input image

sampleImage = loadImage('sample_1.jpg')
print("Loaded Image")

# load the pre-trained Inception model from disk
print("Loading Inception Model...")
baseModel = InceptionV3(include_top=False, weights="imagenet")
print("Loaded Inception Model")

Loaded Image
Loading Inception Model...


2023-03-15 13:05:27.371810: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


87910968/87910968 [==============================] - 7s 0us/step
Loaded Inception Model


In [10]:
# Explore the layers you can use by looking at the model summary
# try using different mixed layers
baseModel.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d[0][0]']                 
 alization)                     32)                                                    

                                                                                                  
 average_pooling2d (AveragePool  (None, None, None,   0          ['max_pooling2d_1[0][0]']        
 ing2D)                         192)                                                              
                                                                                                  
 conv2d_5 (Conv2D)              (None, None, None,   12288       ['max_pooling2d_1[0][0]']        
                                64)                                                               
                                                                                                  
 conv2d_7 (Conv2D)              (None, None, None,   76800       ['activation_6[0][0]']           
                                64)                                                               
                                                                                                  
 conv2d_10

                                                                                                  
 conv2d_18 (Conv2D)             (None, None, None,   16384       ['average_pooling2d_1[0][0]']    
                                64)                                                               
                                                                                                  
 batch_normalization_12 (BatchN  (None, None, None,   192        ['conv2d_12[0][0]']              
 ormalization)                  64)                                                               
                                                                                                  
 batch_normalization_14 (BatchN  (None, None, None,   192        ['conv2d_14[0][0]']              
 ormalization)                  64)                                                               
                                                                                                  
 batch_nor

                                                                                                  
 batch_normalization_25 (BatchN  (None, None, None,   192        ['conv2d_25[0][0]']              
 ormalization)                  64)                                                               
                                                                                                  
 activation_19 (Activation)     (None, None, None,   0           ['batch_normalization_19[0][0]'] 
                                64)                                                               
                                                                                                  
 activation_21 (Activation)     (None, None, None,   0           ['batch_normalization_21[0][0]'] 
                                64)                                                               
                                                                                                  
 activatio

                                128)                                                              
                                                                                                  
 conv2d_36 (Conv2D)             (None, None, None,   114688      ['activation_35[0][0]']          
                                128)                                                              
                                                                                                  
 batch_normalization_31 (BatchN  (None, None, None,   384        ['conv2d_31[0][0]']              
 ormalization)                  128)                                                              
                                                                                                  
 batch_normalization_36 (BatchN  (None, None, None,   384        ['conv2d_36[0][0]']              
 ormalization)                  128)                                                              
          

...

 batch_normalization_53 (BatchN  (None, None, None,   576        ['conv2d_53[0][0]']              
 ormalization)                  192)                                                              
                                                                                                  
 batch_normalization_58 (BatchN  (None, None, None,   576        ['conv2d_58[0][0]']              
 ormalization)                  192)                                                              
                                                                                                  
 batch_normalization_59 (BatchN  (None, None, None,   576        ['conv2d_59[0][0]']              
 ormalization)                  192)                                                              
                                                                                                  
 activation_50 (Activation)     (None, None, None,   0           ['batch_normalization_50[0][0]'] 
          

 conv2d_60 (Conv2D)             (None, None, None,   147456      ['mixed6[0][0]']                 
                                192)                                                              
                                                                                                  
 conv2d_63 (Conv2D)             (None, None, None,   258048      ['activation_62[0][0]']          
                                192)                                                              
                                                                                                  
 conv2d_68 (Conv2D)             (None, None, None,   258048      ['activation_67[0][0]']          
                                192)                                                              
                                                                                                  
 conv2d_69 (Conv2D)             (None, None, None,   147456      ['average_pooling2d_6[0][0]']    
          

 batch_normalization_71 (BatchN  (None, None, None,   960        ['conv2d_71[0][0]']              
 ormalization)                  320)                                                              
                                                                                                  
 batch_normalization_75 (BatchN  (None, None, None,   576        ['conv2d_75[0][0]']              
 ormalization)                  192)                                                              
                                                                                                  
 activation_71 (Activation)     (None, None, None,   0           ['batch_normalization_71[0][0]'] 
                                320)                                                              
                                                                                                  
 activation_75 (Activation)     (None, None, None,   0           ['batch_normalization_75[0][0]'] 
          

                                384)                                                              
                                                                                                  
 activation_79 (Activation)     (None, None, None,   0           ['batch_normalization_79[0][0]'] 
                                384)                                                              
                                                                                                  
 activation_82 (Activation)     (None, None, None,   0           ['batch_normalization_82[0][0]'] 
                                384)                                                              
                                                                                                  
 activation_83 (Activation)     (None, None, None,   0           ['batch_normalization_83[0][0]'] 
                                384)                                                              
          

 ormalization)                  384)                                                              
                                                                                                  
 batch_normalization_92 (BatchN  (None, None, None,   1152       ['conv2d_92[0][0]']              
 ormalization)                  384)                                                              
                                                                                                  
 conv2d_93 (Conv2D)             (None, None, None,   393216      ['average_pooling2d_8[0][0]']    
                                192)                                                              
                                                                                                  
 batch_normalization_85 (BatchN  (None, None, None,   960        ['conv2d_85[0][0]']              
 ormalization)                  320)                                                              
          

In [11]:
# build our dreaming model
layers = [baseModel.get_layer(name).output for name in names]
deepDreamModel = tf.keras.Model(inputs=baseModel.input, outputs=layers)

# We convert the image to a TensorFlow constant for better performance,
image = tf.constant(sampleImage)

# We take the first two dimensions of the image and cast then them to float
baseShape = tf.cast(tf.shape(image)[:-1], tf.float32)

# Run our Deep Dream Generator!



In [ ]:
import matplotlib.pyplot as plt

# We iterate over the number of Octaves (sizes) we are going to create using Deep Dream
for n in range(NUM_OCTAVES):

    # get the width and height for the current octave and cast them to integers
    print("Working on octave {}".format(n))
    newShape = tf.cast(baseShape * (OCTAVE_SCALE ** n), tf.int32)

    # Resize the image with new shape and convert it to numpy before running 
    # through our DeepDream Model
    image = tf.image.resize(image, newShape).numpy()
    image = fullDeepDreamModel(model=deepDreamModel, image=image, iterations=1000, stepSize=0.0001)

# convert the final image to a numpy array and then save it to disk
finalImage = np.array(image)
cv2.imwrite('output.jpg', finalImage)

Working on octave 0
Iteration 0 with loss 1.3065639734268188
Iteration 50 with loss 1.4815843105316162
Iteration 100 with loss 1.6272493600845337
Iteration 150 with loss 1.750020980834961
Iteration 200 with loss 1.854102373123169
Iteration 250 with loss 1.9434443712234497
Iteration 300 with loss 2.021371603012085
Iteration 350 with loss 2.0916671752929688
Iteration 400 with loss 2.1541130542755127
Iteration 450 with loss 2.20997953414917
Iteration 500 with loss 2.261092185974121
Iteration 550 with loss 2.3087105751037598
Iteration 600 with loss 2.351649284362793
Iteration 650 with loss 2.39180064201355
Iteration 700 with loss 2.4287540912628174
Iteration 750 with loss 2.4636120796203613
Iteration 800 with loss 2.4954793453216553
Iteration 850 with loss 2.525620698928833
Iteration 900 with loss 2.554154634475708
Iteration 950 with loss 2.581627607345581
Working on octave 1
Iteration 0 with loss 1.517755150794983
Iteration 50 with loss 1.7361654043197632
Iteration 100 with loss 1.8850865

## Please experiment with diferent Octaves, Iterations, Layers and Octave Scales!

Start making your own 'Art'